In [1]:
# Core
import numpy as np
import pandas as pd

# Tensorflow / ML
import tensorflow as tf
from tensorflow.python.keras import backend as K

# Plotting
import matplotlib.pyplot as plt
import matplotlib as mpl

# Utility
import time

# MSE Imports
import kepler_sieve
from asteroid_element import load_ast_elt
from candidate_element import asteroid_elts, perturb_elts, random_elts
from ztf_ast import load_ztf_nearest_ast, calc_hit_freq
from ztf_element import load_ztf_batch, make_ztf_batch, ztf_elt_summary
from asteroid_model import AsteroidPosition, AsteroidDirection, make_model_ast_pos
from asteroid_search_layers import CandidateElements, TrajectoryScore
from asteroid_search_model import AsteroidSearchModel, make_adam_opt
from asteroid_search_report import traj_diff
from astro_utils import deg2dist, dist2deg, dist2sec
from tf_utils import Identity

Found 4 GPUs.  Setting memory growth = True.


In [2]:
# Aliases
keras = tf.keras

# Constants
dtype = tf.float32
dtype_np = np.float32
space_dims = 3

In [3]:
# Set plot style variables
mpl.rcParams['figure.figsize'] = [16.0, 10.0]
mpl.rcParams['font.size'] = 16

## Load ZTF Data and Batch of Orbital Elements

In [4]:
# Load orbital elements for known asteroids
ast_elt = load_ast_elt()

# Number of asteroids
N_ast = ast_elt.shape[0]

In [5]:
# Load ztf nearest asteroid data
ztf_ast = load_ztf_nearest_ast()

In [6]:
# Asteroid numbers and hit counts
ast_num, hit_count = calc_hit_freq(ztf=ztf_ast, thresh_sec=2.0)

# Sort the hit counts in descending order and find the top batch_size
idx = np.argsort(hit_count)[::-1]

# Extract the asteroid number and hit count for this batch
ast_num_best = ast_num[idx]
hit_count_best = hit_count[idx]

In [7]:
# Parameters to build elements batch
batch_size = 64
h = 1.0/64.0  # (1.5625%)
R_deg = 1.0

# Batch of unperturbed elements
elts_ast = asteroid_elts(ast_nums=ast_num_best[0:batch_size])

In [8]:
# Review unperturbed elements
elts_ast

,element_id,a,e,inc,Omega,omega,f,epoch,h,R
0,51921,2.669306,0.217361,0.499554,4.699703,2.450796,-1.133491,58600.0,0.015625,0.017453
1,59244,2.634727,0.262503,0.465045,5.738298,1.766995,-1.601363,58600.0,0.015625,0.017453
2,15786,1.883227,0.047655,0.392360,6.134689,0.804823,-1.246069,58600.0,0.015625,0.017453
3,3904,2.556387,0.098279,0.261542,5.450163,2.202423,-1.357345,58600.0,0.015625,0.017453
4,142999,2.619944,0.191376,0.514017,0.238022,0.946463,-1.299301,58600.0,0.015625,0.017453
...,...,...,...,...,...,...,...,...,...,...
59,11952,2.219650,0.086091,0.117967,0.042442,2.904823,-3.016580,58600.0,0.015625,0.017453
60,134815,2.612770,0.140831,0.513922,0.272689,0.645552,-0.957836,58600.0,0.015625,0.017453
61,27860,2.619406,0.096185,0.200633,5.541400,3.266046,3.948770,58600.0,0.015625,0.017453
62,85937,2.342291,0.197267,0.439063,5.279693,3.210025,3.947687,58600.0,0.015625,0.017453


In [9]:
# Inpute to perturb elements: large
sigma_a = 0.05
sigma_e = 0.01
sigma_inc_deg = 0.25
sigma_f_deg = 1.0
sigma_Omega_deg = 1.0
sigma_omega_deg = 1.0
mask_pert = None
random_seed = 42

# Perturb orbital elements
elts_pert = perturb_elts(elts_ast, sigma_a=sigma_a, sigma_e=sigma_e, 
                         sigma_inc_deg=sigma_inc_deg, sigma_f_deg=sigma_f_deg, 
                         sigma_Omega_deg=sigma_Omega_deg, sigma_omega_deg=sigma_omega_deg,
                         mask_pert=mask_pert, random_seed=random_seed)

In [10]:
# Review perturbed elements
elts_pert

,element_id,a,e,inc,Omega,omega,f,epoch,h,R
0,51921,2.736430,0.219134,0.499988,4.721815,2.452489,-1.129754,58600.0,0.015625,0.017453
1,59244,2.616575,0.266087,0.462848,5.725946,1.777382,-1.623105,58600.0,0.015625,0.017453
2,15786,1.945213,0.047621,0.385594,6.142435,0.790543,-1.243047,58600.0,0.015625,0.017453
3,3904,2.758664,0.099270,0.261841,5.463683,2.238942,-1.350620,58600.0,0.015625,0.017453
4,142999,2.589450,0.192070,0.509382,0.221844,0.928905,-1.314727,58600.0,0.015625,0.017453
...,...,...,...,...,...,...,...,...,...,...
59,11952,2.330603,0.084892,0.117649,0.042808,2.890716,-3.000560,58600.0,0.015625,0.017453
60,134815,2.550916,0.141660,0.510228,0.284591,0.630896,-0.920797,58600.0,0.015625,0.017453
61,27860,2.595202,0.098315,0.194023,5.535984,3.255585,3.966790,58600.0,0.015625,0.017453
62,85937,2.216242,0.195323,0.437115,5.285351,3.172956,3.921169,58600.0,0.015625,0.017453


In [11]:
# Random elements
elts_rand = random_elts(element_id_start=0, size=batch_size, h=h, R_deg=R_deg,
                        random_seed=random_seed, dtype=dtype_np)

In [12]:
# Review random elements
elts_rand

,element_id,a,e,f,inc,Omega,omega,epoch,h,R
0,0,2.346512,0.191774,3.947884,0.123224,5.077989,4.698056,58600.0,0.015625,0.017453
1,1,3.002211,0.239903,-2.104391,0.032646,3.623584,5.988327,58600.0,0.015625,0.017453
2,2,2.317087,0.055763,-4.869156,0.034794,3.476631,2.078166,58600.0,0.015625,0.017453
3,3,2.349419,0.200190,3.323487,0.222124,1.726133,3.473125,58600.0,0.015625,0.017453
4,4,2.712220,0.116107,4.183110,0.200886,0.668298,3.595820,58600.0,0.015625,0.017453
...,...,...,...,...,...,...,...,...,...,...
59,59,2.328723,0.639718,1.285795,0.064922,1.580547,4.395165,58600.0,0.015625,0.017453
60,60,3.972593,0.039944,-0.006957,0.176738,5.749878,0.869298,58600.0,0.015625,0.017453
61,61,3.019390,0.032757,-3.315554,0.115168,1.725482,0.834064,58600.0,0.015625,0.017453
62,62,2.481321,0.025182,2.025885,0.337006,2.972029,6.091780,58600.0,0.015625,0.017453


## Batches of ZTF Data vs. Elements

In [13]:
# Arguments to make_ztf_batch
# thresh_deg = 1.0
thresh_deg = 4.0
near_ast = False
regenerate = False

In [14]:
# Load unperturbed element batch
ztf_elt_ast = load_ztf_batch(elts=elts_ast, thresh_deg=thresh_deg, near_ast=near_ast, regenerate=regenerate)

In [15]:
# Load perturbed element batch
ztf_elt_pert = load_ztf_batch(elts=elts_pert, thresh_deg=thresh_deg, near_ast=near_ast, regenerate=regenerate)

In [16]:
# Load random element batch
ztf_elt_rand = load_ztf_batch(elts=elts_rand, thresh_deg=thresh_deg, near_ast=near_ast, regenerate=regenerate)

In [17]:
# Summarize the ztf element batch: unperturbed asteroids
score_by_elt_ast = ztf_elt_summary(ztf_elt_ast, 'Unperturbed Asteroids')

ZTF Element Dataframe Unperturbed Asteroids:
                  Total     (Per Batch)
Observations   :  1233691   (    19276)
Hits           :    10333   (   161.45)

Summarize score = sum(-1.0 - log(v)) by batch.  (Mean=0, Variance=num_obs)
Mean score     :    3347.31
Sqrt(batch_obs):     138.84
Mean t_score   :      26.13


In [18]:
# score_by_elt_ast

In [19]:
# Summarize the ztf element batch: perturbed asteroids
score_by_elt_pert = ztf_elt_summary(ztf_elt_pert, 'Perturbed Asteroids')

ZTF Element Dataframe Perturbed Asteroids:
                  Total     (Per Batch)
Observations   :  1148437   (    17944)
Hits           :        0   (     0.00)

Summarize score = sum(-1.0 - log(v)) by batch.  (Mean=0, Variance=num_obs)
Mean score     :     146.79
Sqrt(batch_obs):     133.96
Mean t_score   :       0.90


In [20]:
# Summarize the ztf element batch: random elements
score_by_elt_rand = ztf_elt_summary(ztf_elt_rand, 'Random Elements')

ZTF Element Dataframe Random Elements:
                  Total     (Per Batch)
Observations   :  1056703   (    16773)
Hits           :        0   (     0.00)

Summarize score = sum(-1.0 - log(v)) by batch.  (Mean=0, Variance=num_obs)
Mean score     :      82.21
Sqrt(batch_obs):     129.51
Mean t_score   :       0.28


## View Example DataFrames and Hits

In [21]:
ztf_elt_ast

,ztf_id,element_id,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,...,vz,elt_ux,elt_uy,elt_uz,elt_r,s,s_sec,z,v,is_hit
0,25248,733,b'ZTF18absqzef',611130485415015022,7576,58365.130486,269.830331,-14.496884,-0.002867,-0.987841,...,0.003988,-0.053007,-0.979529,0.194197,2.419761,0.063916,13185.789822,0.997957,0.838525,False
1,37580,733,b'ZTF18abcsqhf',618126363115010066,8913,58372.126366,271.256633,-11.466062,0.021493,-0.978028,...,0.003945,-0.042686,-0.979082,0.198938,2.501611,0.064738,13355.474480,0.997905,0.860237,False
2,37581,733,b'ZTF18abcsqhf',617209183115010025,8768,58371.209190,271.256646,-11.466037,0.021493,-0.978028,...,0.003950,-0.044314,-0.979129,0.198349,2.490799,0.066430,13704.750780,0.997794,0.905803,False
3,37582,733,b'ZTF18abcsqhf',617122523115010032,8730,58371.122523,271.256648,-11.466060,0.021493,-0.978028,...,0.003951,-0.044456,-0.979134,0.198292,2.489770,0.066579,13735.393413,0.997784,0.909857,False
4,37587,733,b'ZTF18abcsqhf',567274573115010023,3341,58321.274572,271.256647,-11.466069,0.021493,-0.978028,...,0.004191,-0.000053,-0.989617,0.143727,2.018643,0.068175,14064.801909,0.997676,0.954004,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233686,5651378,324582,b'ZTF20aaqvkyo',1150185754815015007,96635,58904.185752,42.122572,29.195466,0.647485,0.731228,...,-0.001541,0.627608,0.750726,0.206199,2.981918,0.029094,6001.222051,0.999577,0.173740,False
1233687,5651434,324582,b'ZTF20aaqvkus',1150185310315015002,96634,58904.185313,43.256211,29.954124,0.631015,0.743348,...,-0.001541,0.627610,0.750725,0.206200,2.981912,0.017707,3652.453216,0.999843,0.064359,False
1233688,5651513,324582,b'ZTF20aaqvlna',1150185755115015011,96635,58904.185752,42.388038,28.228923,0.650751,0.733102,...,-0.001541,0.627608,0.750726,0.206199,2.981918,0.030306,6251.306560,0.999541,0.188521,False
1233689,5651704,324582,b'ZTF20aaqvlmz',1150185755115015003,96635,58904.185752,41.916630,28.631276,0.653128,0.728579,...,-0.001541,0.627608,0.750726,0.206199,2.981918,0.033791,6970.130701,0.999429,0.234365,False


In [22]:
# Review hits
mask = ztf_elt_ast.is_hit
ztf_elt_ast[mask]

,ztf_id,element_id,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,...,vz,elt_ux,elt_uy,elt_uz,elt_r,s,s_sec,z,v,is_hit
178,341737,733,b'ZTF19abizrac',937427766115015019,40797,58691.427766,33.130412,43.596186,0.606481,0.637452,...,-0.002127,0.606486,0.637448,0.475220,2.115260,0.000007,1.509138,1.0,1.098777e-08,True
191,345725,733,b'ZTF19abjajfg',937467364815015020,40840,58691.467361,33.148426,43.605278,0.606265,0.637618,...,-0.002129,0.606270,0.637614,0.475274,2.114851,0.000007,1.479386,1.0,1.055879e-08,True
196,346522,733,b'ZTF19abjajmr',937468726115015011,40842,58691.468727,33.149062,43.605587,0.606257,0.637624,...,-0.002129,0.606263,0.637619,0.475276,2.114837,0.000007,1.503609,1.0,1.090740e-08,True
205,347644,733,b'ZTF19abiyxiu',937402264815015008,40777,58691.402269,33.118785,43.590288,0.606621,0.637345,...,-0.002127,0.606626,0.637341,0.475186,2.115523,0.000007,1.491440,1.0,1.073157e-08,True
222,431445,733,b'ZTF19abkkfhr',934448315015015003,40221,58688.448310,31.751906,42.913068,0.622775,0.624451,...,-0.002040,0.622780,0.624447,0.471393,2.145603,0.000007,1.493960,1.0,1.076786e-08,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233374,5447574,324582,b'ZTF20aapdfmj',1145121244815015014,95204,58899.121250,42.392273,29.202629,0.644677,0.734024,...,-0.001502,0.644680,0.734021,0.213544,2.915471,0.000005,0.934688,1.0,4.214873e-09,True
1233417,5450145,324582,b'ZTF20aapdfmj',1145121716115015016,95205,58899.121713,42.392439,29.202638,0.644675,0.734026,...,-0.001502,0.644678,0.734022,0.213543,2.915477,0.000005,1.005771,1.0,4.880340e-09,True
1233479,5461311,324582,b'ZTF20aapeobw',1145164884815015018,95284,58899.164884,42.405891,29.202386,0.644538,0.734164,...,-0.001503,0.644541,0.734161,0.213479,2.916052,0.000004,0.890844,1.0,3.828730e-09,True
1233480,5461312,324582,b'ZTF20aapeobw',1145165336115015009,95285,58899.165336,42.406082,29.202412,0.644536,0.734166,...,-0.001503,0.644540,0.734163,0.213478,2.916058,0.000005,1.056357,1.0,5.383601e-09,True


In [23]:
ztf_elt_ast.columns

Index(['ztf_id', 'element_id', 'ObjectID', 'CandidateID', 'TimeStampID', 'mjd',
       'ra', 'dec', 'ux', 'uy', 'uz', 'qx', 'qy', 'qz', 'vx', 'vy', 'vz',
       'elt_ux', 'elt_uy', 'elt_uz', 'elt_r', 's', 's_sec', 'z', 'v',
       'is_hit'],
      dtype='object')

In [24]:
# Alias ztf_elt_ast to ztf_elt
ztf_elt = ztf_elt_ast

In [25]:
# Build numpy array of times
ts_np = ztf_elt.mjd.values.astype(dtype_np)

# Get observation count per element
row_lengths_np = ztf_elt.element_id.groupby(ztf_elt.element_id).count().values.astype(np.int32)

In [26]:
# Review results
element_id_best = ast_num_best[0]
mask = (ztf_elt.element_id == element_id_best)
hits_best = np.sum(ztf_elt[mask].is_hit)
hit_rate_best = np.mean(ztf_elt[mask].is_hit)
rows_best = np.sum(mask)
s_sec_min = np.min(ztf_elt[mask].s_sec)
idx = np.argmin(ztf_elt.s)
ztf_id = ztf_elt.ztf_id[idx]
# ztf_elt[mask].iloc[idx:idx+1]
print(f'Best asteroid has element_id = {element_id_best}')
print(f'Hit count: {hits_best} / {rows_best} observations')
print(f'Hit rate : {hit_rate_best:8.6f}')
print(f'Closest hit: {s_sec_min:0.3f} arc seconds')
# ztf_elt[mask]

Best asteroid has element_id = 51921
Hit count: 158 / 10110 observations
Hit rate : 0.015628
Closest hit: 0.381 arc seconds


## Build Asteroid Search Model

In [27]:
# Additional arguments for asteroid search models
site_name = 'palomar'

# Training parameters
learning_rate = 2.0**-13 # (1.22E-4)
clipnorm = 1.0
regenerate = False

In [28]:
# Review candidate elements
elts_ast

,element_id,a,e,inc,Omega,omega,f,epoch,h,R
0,51921,2.669306,0.217361,0.499554,4.699703,2.450796,-1.133491,58600.0,0.015625,0.017453
1,59244,2.634727,0.262503,0.465045,5.738298,1.766995,-1.601363,58600.0,0.015625,0.017453
2,15786,1.883227,0.047655,0.392360,6.134689,0.804823,-1.246069,58600.0,0.015625,0.017453
3,3904,2.556387,0.098279,0.261542,5.450163,2.202423,-1.357345,58600.0,0.015625,0.017453
4,142999,2.619944,0.191376,0.514017,0.238022,0.946463,-1.299301,58600.0,0.015625,0.017453
...,...,...,...,...,...,...,...,...,...,...
59,11952,2.219650,0.086091,0.117967,0.042442,2.904823,-3.016580,58600.0,0.015625,0.017453
60,134815,2.612770,0.140831,0.513922,0.272689,0.645552,-0.957836,58600.0,0.015625,0.017453
61,27860,2.619406,0.096185,0.200633,5.541400,3.266046,3.948770,58600.0,0.015625,0.017453
62,85937,2.342291,0.197267,0.439063,5.279693,3.210025,3.947687,58600.0,0.015625,0.017453


In [29]:
# Build asteroid search model
model = AsteroidSearchModel(
        elts=elts_ast, ztf_elt=ztf_elt, 
        site_name=site_name, thresh_deg=thresh_deg, 
        learning_rate=learning_rate, clipnorm=clipnorm)

In [30]:
# Dummy inputs for search model; any array with shape [batch_size,] is good
x = tf.ones(batch_size)

In [31]:
# Run model on unperturbed elements
log_like, orbital_elements, mixture_parameters = model(x)

In [32]:
# Summarize log likelihood on unperturbed elements
log_like_tot = np.sum(log_like)
log_like_mean = np.mean(log_like)
log_like_std = np.std(log_like)

# Report on unperturbed elements
print(f'Log likelihood:')
print(f'Total: {log_like_tot:8.2f}')
print(f'Mean: {log_like_mean:8.2f}')
print(f'Std : {log_like_std:8.2f}')
print(f'First 5:')
print(log_like[0:5].numpy())

Log likelihood:
Total:    77.20
Mean:     1.21
Std :     1.45
First 5:
[2.0169735 1.4916104 0.6803085 0.8213857 0.2282602]


In [33]:
# Fite one batch
hist = model.fit(x)

Train on 64 samples
64/64 [==============================] - 2s 37ms/sample - loss: -162.2950


In [34]:
# Evaluate
model.evaluate(x)

64/64 [==============================] - 1s 9ms/sample - loss: -282.7591


-282.75914001464844

In [35]:
# Built in log likelihood caclulation
model.calc_log_like()

<tf.Tensor: shape=(64,), dtype=float32, numpy=
array([ 2.758985  ,  2.2057872 ,  5.506832  ,  3.148364  ,  2.155534  ,
        2.9356306 ,  3.3185585 , 11.96782   ,  2.2630053 ,  6.747738  ,
        5.538798  ,  5.0371456 ,  3.2406807 ,  4.344903  ,  7.020876  ,
        2.1994715 ,  2.229331  ,  6.5643983 ,  7.1962028 ,  6.5121727 ,
        3.5864105 ,  2.7180512 , 11.726983  , 12.061548  ,  6.3269663 ,
        0.27665344,  1.816589  ,  2.6076283 ,  2.9798281 ,  8.382397  ,
        4.2055435 ,  5.5597887 , 10.132792  ,  0.7299549 ,  6.489115  ,
        0.6468203 ,  3.624529  ,  3.7305686 ,  8.510661  ,  2.5122354 ,
        1.0794793 , -1.30963   ,  3.5631318 ,  3.0396473 ,  5.3942394 ,
        3.0356808 ,  7.430238  ,  0.82332355,  2.0394053 ,  6.2325788 ,
        2.2690406 ,  4.2238636 ,  2.3937182 ,  8.254661  ,  6.509577  ,
        0.62242454,  3.927779  ,  3.3142645 ,  0.7262423 , 13.470556  ,
        0.97716457,  7.5968738 ,  0.40148652,  5.2260265 ], dtype=float32)>

In [36]:
model.summary()

Model: "asteroid_search_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
candidate_elements (Candidat multiple                  576       
_________________________________________________________________
direction (AsteroidDirection multiple                  7402146   
_________________________________________________________________
score (TrajectoryScore)      multiple                  0         
_________________________________________________________________
model_asteroid_pos (Model)   ((64, None, 3), (64, None 7402146   
Total params: 14,804,868
Trainable params: 512
Non-trainable params: 14,804,356
_________________________________________________________________


## Fit Model on Unperturbed Elements

In [37]:
# Build asteroid search model
model_ast = AsteroidSearchModel(
                elts=elts_ast, ztf_elt=ztf_elt, 
                site_name=site_name, thresh_deg=thresh_deg, 
                learning_rate=learning_rate, clipnorm=clipnorm)

In [38]:
# Adaptive search parameters
max_batches = 20000
batches_per_epoch = 100
epochs_per_episode = 5
# min_learning_rate = 2.0**-20 # about 9.54E-7
min_learning_rate = 2.0**-22
verbose = 1

# # Tiny size for fast testing
# max_batches = 100
# batches_per_epoch = 10

In [39]:
# alias model_ast to model for interactive testing
# model = model_ast

In [40]:
# Regenerate elements from model
elts_df = model.load_candidates(verbose=True)

Loading candidate elements DataFrame from ../data/candidate_elt/candidate_elt_6490016596481062962.h5.


In [41]:
# Report elements from model
elts_df = model.candidates_df()

# Review regenerated elements and scores
# elts_df

In [42]:
# Learning rate for big training
learning_rate = 2.0**-15

In [43]:
# Train unperturbed model
model_ast.search_adaptive(
    max_batches=max_batches, 
    batches_per_epoch=batches_per_epoch,
    epochs_per_episode=epochs_per_episode,
    min_learning_rate=min_learning_rate,
    regenerate=True,
    verbose=verbose)

Train on 6400 samples
Epoch 1/5
6400/6400 [==============================] - 7s 1ms/sample - loss: -716.9449
Epoch 2/5
6400/6400 [==============================] - 5s 817us/sample - loss: -1750.4615
Epoch 3/5
6400/6400 [==============================] - 5s 833us/sample - loss: -3660.3257
Epoch 4/5
6400/6400 [==============================] - 5s 818us/sample - loss: -7147.9408
Epoch 5/5
6400/6400 [==============================] - 5s 805us/sample - loss: -12830.5685
Log Likelihood: 16402.48

Training episode 2: Epoch    5
learning_rate=1.221e-04, total training time 29 sec.
Train on 6400 samples
Epoch 6/10
6400/6400 [==============================] - 5s 807us/sample - loss: -20007.3165
Epoch 7/10
6400/6400 [==============================] - 5s 810us/sample - loss: -26199.4981
Epoch 8/10
6400/6400 [==============================] - 5s 812us/sample - loss: -29818.9164
Epoch 9/10
6400/6400 [==============================] - 5s 816us/sample - loss: -31425.5506
Epoch 10/10
6400/6400 [=======

In [44]:
# # Second (tune-up) adaptive training
# model.search_adaptive(
#     max_batches=max_batches, 
#     batches_per_epoch=batches_per_epoch,
#     epochs_per_episode=epochs_per_episode,
#     min_learning_rate=min_learning_rate,
#     regenerate=True,
#     verbose=verbose)

In [45]:
model_ast.load_candidates()

In [46]:
model_ast.current_loss()

-32223.607421875

In [47]:
# Review likelihoods by element
log_like_ast, orbital_elements_ast, mixture_parameters_ast = model_ast.calc_outputs()

In [48]:
log_like_ast

<tf.Tensor: shape=(64,), dtype=float32, numpy=
array([ 392.41013 ,  236.43759 ,  636.95197 ,  519.46375 ,  275.02908 ,
        340.53137 ,  332.44995 , 1372.0034  ,  103.77487 ,  666.11334 ,
        406.2512  ,  994.3869  ,  163.99779 ,  347.49194 , 1214.498   ,
        169.82951 ,  152.82843 ,  734.5538  ,  762.15326 ,  562.2839  ,
        253.31468 ,  176.46133 , 1109.5739  ,  636.44653 ,  275.1175  ,
        290.2945  ,  311.60852 ,  512.68353 ,  121.58192 , 1036.1904  ,
        217.42603 ,  774.5002  ,  594.06586 ,   81.84558 ,  936.6492  ,
        143.59726 ,  451.77896 ,   76.93962 ,  603.13116 ,  326.9774  ,
        130.91602 ,  672.82684 ,  420.41394 ,  378.5126  ,  364.65707 ,
        265.36975 , 1220.2821  ,  488.19308 ,  739.42505 , 1352.6709  ,
        224.08293 ,  394.63263 ,  485.54944 ,  349.96884 , 1035.4465  ,
         25.544394,  374.95486 ,  198.4577  ,  562.2462  , 1641.5841  ,
        195.47778 ,  558.8929  ,  135.62494 ,  694.2476  ], dtype=float32)>

In [49]:
# Review resolution in arc seconds
dist2deg(mixture_parameters_ast[2])

array([0.38687357, 0.3702561 , 0.33875394, 0.3691627 , 0.4130769 ,
       0.37055847, 0.3355942 , 0.35776415, 0.47727972, 0.41547525,
       0.3545998 , 0.3035949 , 0.38442284, 0.3896837 , 0.34934172,
       0.42628533, 0.40763766, 0.33405513, 0.3430481 , 0.3775206 ,
       0.42609715, 0.37008998, 0.39970317, 0.43094313, 0.43115813,
       0.36327323, 0.37638944, 0.3704966 , 0.45765957, 0.3972123 ,
       0.4072916 , 0.34733656, 0.4310169 , 0.48065785, 0.34165707,
       0.40656742, 0.3757989 , 0.53055173, 0.4253696 , 0.37008977,
       0.46931678, 0.38395038, 0.34720853, 0.33800995, 0.3447405 ,
       0.41793698, 0.38053945, 0.27486542, 0.37839776, 0.36440974,
       0.42247894, 0.33939824, 0.3381194 , 0.42811403, 0.4027686 ,
       0.61473495, 0.27397743, 0.42785913, 0.4358731 , 0.34387973,
       0.44057932, 0.35391822, 0.45486757, 0.34339043], dtype=float32)

In [50]:
elts_fit = model_ast.candidates_df()
elts_fit

,element_id,a,e,inc,Omega,omega,f,epoch,h,lam,R,R_deg,R_sec,log_like
0,51921,2.804808,0.259112,0.512393,4.672254,2.466877,-1.143883,58600.0,0.201922,3.340585,0.006752,0.386874,1392.744873,392.409180
1,59244,2.531739,0.249448,0.450170,5.703967,1.624079,-1.519800,58600.0,0.200257,3.647170,0.006462,0.370256,1332.921997,236.437958
2,15786,1.771008,0.041403,0.371486,6.132711,0.721219,-1.273219,58600.0,0.231156,4.357039,0.005912,0.338754,1219.514160,636.952637
3,3904,2.479498,0.122656,0.241301,5.450848,2.383435,-1.585961,58600.0,0.227762,3.668806,0.006443,0.369163,1328.985718,519.463989
4,142999,2.447160,0.171863,0.506247,0.230015,0.905868,-1.329729,58600.0,0.209058,2.930212,0.007210,0.413077,1487.076904,275.029175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,11952,2.241554,0.000000,0.105155,0.102059,2.766834,-3.046987,58600.0,0.217031,4.228118,0.006002,0.343880,1237.967041,1641.586060
60,134815,2.644034,0.173822,0.521250,0.286307,0.625509,-0.996840,58600.0,0.200322,2.575804,0.007690,0.440579,1586.085571,195.477768
61,27860,2.484822,0.156649,0.189595,5.455099,3.327430,3.931279,58600.0,0.183606,3.991668,0.006177,0.353918,1274.105591,558.894165
62,85937,2.449582,0.145876,0.453438,5.215494,3.168766,4.067191,58600.0,0.199136,2.416525,0.007939,0.454868,1637.523315,135.625000


In [51]:
model_pos = make_model_ast_pos(ts_np=ztf_elt.mjd, row_lengths_np=row_lengths_np)

In [52]:
traj_err = traj_diff(elts_ast, elts_fit, model_pos)
traj_err

<tf.Tensor: shape=(64,), dtype=float32, numpy=
array([8.27632204e-04, 9.42678191e-03, 1.34793874e-02, 1.56983919e-02,
       1.32547924e-02, 1.00538495e-03, 1.10044248e-01, 1.87210124e-02,
       8.33000406e-04, 1.25335902e-03, 4.86092828e-03, 1.73757710e-02,
       1.69169868e-03, 7.29689491e-04, 2.66043134e-02, 6.33729948e-03,
       4.93627181e-03, 3.65518965e-03, 6.97506405e-03, 1.62303410e-02,
       2.45733373e-03, 2.65682396e-02, 2.90434645e-03, 1.86909118e-03,
       1.86897367e-02, 1.40942202e-03, 6.65363157e-03, 1.59414345e-03,
       3.09424271e-04, 8.94755009e-04, 4.00706800e-03, 3.56943859e-03,
       3.31121869e-03, 5.13856241e-04, 3.40341181e-02, 4.69606649e-03,
       4.36457107e-03, 6.25240093e-04, 6.61252486e-03, 1.53891928e-02,
       1.36980286e-03, 1.49056818e-02, 5.86119946e-03, 3.19068087e-03,
       7.03936256e-03, 9.65270586e-03, 1.39542148e-02, 4.24517691e-01,
       1.02883885e-02, 1.02639077e-02, 1.19949654e-02, 2.24601966e-03,
       3.93343018e-03, 4.07293

In [53]:
np.mean(traj_err)

0.016089903

In [54]:
# plot log_like

## Train on Perturbed Elements

In [55]:
# Build asteroid search model
model_pert = AsteroidSearchModel(
                 elts=elts_pert, ztf_elt=ztf_elt_pert, site_name=site_name,
                 thresh_deg=thresh_deg, h=h, R_deg=R_deg,
                 learning_rate=learning_rate, clipnorm=clipnorm)

TypeError: ('Keyword argument not understood:', 'h')

In [ ]:
# Train model on perturbed elements
model_pert.search_adaptive(
    max_batches=max_batches, 
    batches_per_epoch=batches_per_epoch,
    epochs_per_episode=epochs_per_episode,
    min_learning_rate=min_learning_rate,
    regenerate=regenerate,
    verbose=verbose)

In [ ]:
# Review likelihoods by element
log_like_per, orbital_elements_pert, mixture_parameters_pert = model_pert.calc_outputs()

In [ ]:
log_like_pert

## Train on Random Elements

In [ ]:
# Filter elts_rand down to only those that had matching ztf observations
idx = np.unique(ztf_elt_rand.element_id)
elts_rand = elts_rand.loc[idx]

In [ ]:
elts_rand['a'].dtype

In [ ]:
elts_rand['a'].dtype

In [ ]:
for col in ['a', 'e', 'f', 'inc', 'Omega', 'omega', 'epoch']:
    elts_rand[col] = elts_rand[col].astype(np.float32)

In [ ]:
# Build asteroid search model
model_rand = AsteroidSearchModel(
                 elts=elts_rand, ztf_elt=ztf_elt_rand, site_name=site_name,
                 thresh_deg=thresh_deg, h=h, R_deg=R_deg,
                 learning_rate=learning_rate, clipnorm=clipnorm)

In [ ]:
# Train model on perturbed elements
model_rand.search_adaptive(
    max_batches=max_batches, 
    batches_per_epoch=batches_per_epoch,
    epochs_per_episode=epochs_per_episode,
    min_learning_rate=min_learning_rate,
    verbose=verbose)

In [ ]:
# Review likelihoods by element
log_like_rand, orbital_elements_rand, mixture_parameters_rand = model_rand.calc_outputs()

In [ ]:
log_like_rand

In [ ]:
# orbital_elements_rand

In [ ]:
from candidate_element import elts_np2df